In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import TimeSeriesSplit

## QUANTILE REGRESSION

In [2]:
data = pd.read_csv('non_nan_df (1).csv', )
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,0.000000,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,0.000000,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [3]:
def create_lags(data, tickers, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs, lagged_tickers

In [4]:
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

data, lagged_covs, lagged_tickers = create_lags(data, tickers, covs, period)

/var/folders/xt/0lxqn7_s79zd4vr4x79lt_l80000gn/T/ipykernel_90650/289628482.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[lagged_column_name] = data[tick].shift(i)
/var/folders/xt/0lxqn7_s79zd4vr4x79lt_l80000gn/T/ipykernel_90650/289628482.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[lagged_column_name] = data[tick].shift(i)
/var/folders/xt/0lxqn7_s79zd4vr4x79lt_l80000gn/T/ipykernel_90650/289628482.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

In [6]:
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,...,ROG_lag3,ROG_lag4,ROG_lag5,ROG_lag6,ROG_lag7,ROG_lag8,ROG_lag9,ROG_lag10,ROG_lag11,ROG_lag12
0,2001-02-01,4.2,176.000,5.160952,-0.195020,-9.538746,4.143180,2.851534,4.509829,0.837527,...,1.748252,13.043478,-9.318996,-2.447552,2.142857,13.131313,-8.163265,7.370518,6.922258,54.440789
1,2001-03-01,4.3,176.100,5.098947,-0.217255,-5.857225,5.615486,-6.323922,-9.828843,-3.186580,...,12.886598,1.748252,13.043478,-9.318996,-2.447552,2.142857,13.131313,-8.163265,7.370518,6.922258
2,2001-04-01,4.4,176.400,4.885455,0.756989,8.836250,-1.512969,-5.723911,10.300706,10.879156,...,-5.022831,12.886598,1.748252,13.043478,-9.318996,-2.447552,2.142857,13.131313,-8.163265,7.370518
3,2001-05-01,4.3,177.300,5.141000,0.461042,-0.560577,0.631283,-2.660517,0.487142,-0.352891,...,-8.846156,-5.022831,12.886598,1.748252,13.043478,-9.318996,-2.447552,2.142857,13.131313,-8.163265
4,2001-06-01,4.5,177.700,5.391364,-0.691442,-2.659796,-2.237527,-3.576694,3.433253,-12.344135,...,-0.112520,-8.846156,-5.022831,12.886598,1.748252,13.043478,-9.318996,-2.447552,2.142857,13.131313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,...,-2.707956,-3.448826,-6.956359,2.731887,-1.239735,-1.972516,-0.360696,-0.476186,0.018314,-0.018311
264,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,...,-53.665066,-2.707956,-3.448826,-6.956359,2.731887,-1.239735,-1.972516,-0.360696,-0.476186,0.018314
265,2023-03-01,3.5,301.808,3.746842,0.000000,3.313488,8.035329,0.507544,1.862736,10.703390,...,9.446071,-53.665066,-2.707956,-3.448826,-6.956359,2.731887,-1.239735,-1.972516,-0.360696,-0.476186
266,2023-04-01,3.4,302.918,3.663043,0.000000,1.985238,5.489119,-3.664707,5.612908,15.269915,...,16.968326,9.446071,-53.665066,-2.707956,-3.448826,-6.956359,2.731887,-1.239735,-1.972516,-0.360696


### Defining tick_loss function

In [50]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss, df

### 1-step-ahead VaR: 1 month

In [59]:
import statsmodels.api as sm
import numpy as np

def one_step_ahead_VaR(data, lagged_covs, lagged_tickers, tickers, alpha,t):
    losses = {}

    for ticker in tickers:
        lagged_tickers_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag')]

        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]
        
        X_train = train_data[lagged_covs + lagged_tickers_ticker]
        y_train = train_data[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        X_test = test_data[lagged_covs + lagged_tickers_ticker]
        VaR_forecast = quantreg_results.predict(X_test)

        r = test_data[ticker]
        loss, df = tick_loss(alpha, r, VaR_forecast)

        losses[ticker] = (loss, df)

    return losses

In [60]:
losses_one_step_ahead = one_step_ahead_VaR(data, lagged_covs, lagged_tickers, tickers, alpha=0.05,t=255)

# Print losses
for ticker, loss_tuple in losses_one_step_ahead.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Ticker: AZN
Tick Loss: 21.634284863486105
        Return        VaR  Indicator
255   0.120491  -1.473235          0
256  -0.616736 -11.369532          0
257   0.242173  -0.580941          0
258  -5.813079  -1.330667          1
259 -11.466275   0.311941          1
260   7.239243 -17.316250          0
261  15.575584   8.317337          0
262  -0.250105  -0.995594          0
263  -3.584079  -2.135434          1
264  -0.290649 -18.443566          0
265   8.035329  -6.386341          0
266   5.489119  -3.528553          0
267  -0.191204  -9.079582          0
Ticker: BMY
Tick Loss: 148.88680579216881
        Return        VaR  Indicator
255   0.239147   0.009884          0
256   2.054343  20.924493          1
257  -3.516240  15.226618          1
258  -8.633766  12.148587          1
259   5.459121   3.428285          0
260   8.974528  -2.499059          0
261   4.416367  35.781718          1
262 -10.376181   0.151922          1
263   0.972908 -19.152221          0
264  -4.328217  27.220082   

### 3-step-ahead VaR: 3 months

In [68]:
import statsmodels.api as sm
import numpy as np

def three_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha, t):
    losses = {}

    for ticker in tickers:
        lagged_covs_t3 = [cov for cov in lagged_covs if int(cov.split('_lag')[1]) >= 3]
        lagged_tickers_t3_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag') and int(tick.split('_lag')[1]) >= 3]
        
        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]
        
        X_train = train_data[lagged_covs_t3 + lagged_tickers_t3_ticker]
        y_train = train_data[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        X_test = test_data[lagged_covs_t3 + lagged_tickers_t3_ticker]
        VaR_forecast = quantreg_results.predict(X_test)

        r = test_data[ticker]
        loss, df = tick_loss(alpha, r, VaR_forecast)

        losses[ticker] = (loss, df)

    return losses

In [76]:
losses_three_step_ahead = three_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha=0.05, t=255)

# Print losses
for ticker, loss_tuple in losses_three_step_ahead.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Ticker: AZN
Tick Loss: 9.591045710588995
        Return        VaR  Indicator
255   0.120491  -9.466200          0
256  -0.616736 -14.533556          0
257   0.242173  -0.496821          0
258  -5.813079  -4.375402          1
259 -11.466275 -12.602060          0
260   7.239243  -6.069612          0
261  15.575584  -0.768018          0
262  -0.250105   0.269485          1
263  -3.584079  -7.895326          0
264  -0.290649 -20.303737          0
265   8.035329  10.642534          1
266   5.489119  -9.639597          0
267  -0.191204 -10.803349          0
Ticker: BMY
Tick Loss: 102.63683531254924
        Return        VaR  Indicator
255   0.239147   0.239147          0
256   2.054343  16.845281          1
257  -3.516240  13.717147          1
258  -8.633766  12.904630          1
259   5.459121   9.654214          1
260   8.974528 -14.921969          0
261   4.416367  22.527424          1
262 -10.376181  -2.025155          1
263   0.972908 -24.271061          0
264  -4.328217   8.850680    

### 6-step-ahead VaR: 6 months

In [70]:
import statsmodels.api as sm
import numpy as np

def six_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha, t):
    losses = {}

    for ticker in tickers:
        lagged_covs_t6 = [cov for cov in lagged_covs if int(cov.split('_lag')[1]) >= 6]
        lagged_tickers_t6_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag') and int(tick.split('_lag')[1]) >= 6]
        
        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]
        
        X_train = train_data[lagged_covs_t6 + lagged_tickers_t6_ticker]
        y_train = train_data[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        X_test = test_data[lagged_covs_t6 + lagged_tickers_t6_ticker]
        VaR_forecast = quantreg_results.predict(X_test)

        r = test_data[ticker]
        loss, df = tick_loss(alpha, r, VaR_forecast)

        losses[ticker] = (loss, df)

    return losses

In [77]:
losses_six_step_ahead = six_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha=0.05, t=255)

# Print losses
for ticker, loss_tuple in losses_six_step_ahead.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

Ticker: AZN
Tick Loss: 12.878092067361845
        Return        VaR  Indicator
255   0.120491  -8.472368          0
256  -0.616736  -5.177132          0
257   0.242173  -2.588849          0
258  -5.813079  -5.976497          0
259 -11.466275  -3.605159          1
260   7.239243  -9.194718          0
261  15.575584   3.352700          0
262  -0.250105 -10.372502          0
263  -3.584079  -4.241907          0
264  -0.290649 -11.071136          0
265   8.035329  -3.596556          0
266   5.489119  -3.446319          0
267  -0.191204 -21.459274          0
Ticker: BMY
Tick Loss: 45.808079803742295
        Return        VaR  Indicator
255   0.239147 -11.181276          0
256   2.054343  15.114756          1
257  -3.516240  -0.521823          1
258  -8.633766   0.563249          1
259   5.459121 -10.343258          0
260   8.974528 -17.816162          0
261   4.416367  -1.956368          0
262 -10.376181  -9.433086          1
263   0.972908 -24.962799          0
264  -4.328217  11.058374   

### 9-step-ahead VaR: 9 months

In [72]:
def nine_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha, t):
    losses = {}

    for ticker in tickers:
        lagged_covs_t9 = [cov for cov in lagged_covs if int(cov.split('_lag')[1]) >= 9]
        lagged_tickers_t9_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag') and int(tick.split('_lag')[1]) >= 9]
        
        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]
        
        X_train = train_data[lagged_covs_t9 + lagged_tickers_t9_ticker]
        y_train = train_data[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        X_test = test_data[lagged_covs_t9 + lagged_tickers_t9_ticker]
        VaR_forecast = quantreg_results.predict(X_test)

        r = test_data[ticker]
        loss, df = tick_loss(alpha, r, VaR_forecast)

        losses[ticker] = (loss, df)

    return losses

In [78]:
losses_nine_step_ahead = nine_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha=0.05, t=255)

# Print losses
for ticker, loss_tuple in losses_nine_step_ahead.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Ticker: AZN
Tick Loss: 8.660456749160419
        Return        VaR  Indicator
255   0.120491 -11.103609          0
256  -0.616736  -4.419837          0
257   0.242173  -5.008560          0
258  -5.813079 -16.236584          0
259 -11.466275 -14.381589          0
260   7.239243  -6.243631          0
261  15.575584  -4.885171          0
262  -0.250105 -12.271873          0
263  -3.584079 -12.510843          0
264  -0.290649 -12.667563          0
265   8.035329 -17.165084          0
266   5.489119 -22.138045          0
267  -0.191204 -19.686932          0
Ticker: BMY
Tick Loss: 18.839000182884515
        Return        VaR  Indicator
255   0.239147   0.239147          1
256   2.054343   0.555648          0
257  -3.516240  -0.650103          1
258  -8.633766 -17.059979          0
259   5.459121  -8.886763          0
260   8.974528 -12.434005          0
261   4.416367  -2.765091          0
262 -10.376181  -8.075881          1
263   0.972908 -13.887849          0
264  -4.328217   5.387075    

### 12-step-ahead VaR: 12 months

In [74]:
def twelve_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha, t):
    losses = {}

    for ticker in tickers:
        lagged_covs_t12 = [cov for cov in lagged_covs if int(cov.split('_lag')[1]) >= 12]
        lagged_tickers_t12_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag') and int(tick.split('_lag')[1]) >= 12]
        
        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]
        
        X_train = train_data[lagged_covs_t12 + lagged_tickers_t12_ticker]
        y_train = train_data[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        X_test = test_data[lagged_covs_t12 + lagged_tickers_t12_ticker]
        VaR_forecast = quantreg_results.predict(X_test)

        r = test_data[ticker]
        loss, df = tick_loss(alpha, r, VaR_forecast)

        losses[ticker] = (loss, df)

    return losses

In [80]:
losses_twelve_step_ahead = twelve_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha=0.05, t=255)

# Print losses
for ticker, loss_tuple in losses_twelve_step_ahead.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

Ticker: AZN
Tick Loss: 10.937373031318508
        Return        VaR  Indicator
255   0.120491  -9.146638          0
256  -0.616736 -10.982974          0
257   0.242173 -12.235317          0
258  -5.813079 -10.487249          0
259 -11.466275  -8.619160          1
260   7.239243 -11.250303          0
261  15.575584 -12.261443          0
262  -0.250105 -12.985948          0
263  -3.584079 -12.436759          0
264  -0.290649 -10.367333          0
265   8.035329 -13.879627          0
266   5.489119 -10.457322          0
267  -0.191204 -12.205276          0
Ticker: BMY
Tick Loss: 11.569411883986604
        Return        VaR  Indicator
255   0.239147  -7.803406          0
256   2.054343  -8.889574          0
257  -3.516240 -12.914250          0
258  -8.633766 -12.056973          0
259   5.459121 -15.722662          0
260   8.974528 -10.691956          0
261   4.416367 -14.754243          0
262 -10.376181  -5.076628          1
263   0.972908  -9.015330          0
264  -4.328217  -7.974843   